In [1]:
!pip install pyldavis==3.4.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [2]:
!pip install bertopic --quiet

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [4]:
import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings('ignore')

### пункт 0

Загрузите датасет с помощью функции fetch_20newsgroups

In [5]:
d = fetch_20newsgroups(remove=('headers', 'footer', 'quotes')).data

Токенизируйте и удалите стоп-слова

In [6]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
tokenizer = RegexpTokenizer(r"\w+")
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'could', 'may', '_', '__', '___', 'be', 'do', 'done', 'nice', 'thank',
                   'think', 'rather', 'lot', 'some', 'lack', 'even', 'also'])

In [9]:
d_tokenized = [tokenizer.tokenize(doc.lower()) for doc in d]
d_rmstop = [[word for word in doc if word not in stop_words] for doc in d_tokenized]
d_prep = [' '.join(doc) for doc in d_rmstop]

# part 1: LDA из sklearn

- Однопоточный, неэффективный.
- Совместим в векторайзерами sklearn
- легко использовать

### пункт 1

1) создайте объект класса CountVectorizer 

2) примените его к вашей коллекции, причем так, чтобы коллекция документов была представлена не всеми словами из словаря, а только 1000 словами, которые наиболее часто встречаются в коллекции

(для этого надо воспользоваться методом fit_transform и указать соответствующий параметр)

In [9]:
cv = CountVectorizer()

cv.fit(d)
bag_of_words = cv.transform(d_prep)
sum_words = bag_of_words.sum(axis=0) 
transformed_dataset = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
transformed_dataset = sorted(transformed_dataset, key = lambda x: x[1], reverse=True)

transformed_dataset = transformed_dataset[:1000]

In [10]:
transformed_dataset

[('ax', 62400),
 ('one', 6813),
 ('max', 4610),
 ('people', 4312),
 ('like', 4286),
 ('know', 3727),
 ('get', 3717),
 ('time', 3124),
 ('com', 3065),
 ('new', 2887),
 ('good', 2682),
 ('well', 2633),
 ('us', 2381),
 ('two', 2330),
 ('first', 2329),
 ('see', 2298),
 ('many', 2184),
 ('way', 2181),
 ('much', 2163),
 ('make', 2142),
 ('god', 2115),
 ('system', 2091),
 ('say', 1976),
 ('used', 1945),
 ('right', 1932),
 ('work', 1878),
 ('want', 1848),
 ('said', 1835),
 ('10', 1778),
 ('go', 1767),
 ('file', 1760),
 ('anyone', 1723),
 ('need', 1700),
 ('please', 1685),
 ('something', 1674),
 ('information', 1635),
 ('mail', 1609),
 ('problem', 1604),
 ('since', 1572),
 ('back', 1535),
 ('still', 1508),
 ('thanks', 1507),
 ('year', 1505),
 ('really', 1499),
 ('using', 1492),
 ('going', 1475),
 ('years', 1471),
 ('believe', 1447),
 ('program', 1442),
 ('find', 1436),
 ('last', 1401),
 ('must', 1399),
 ('take', 1390),
 ('space', 1388),
 ('available', 1379),
 ('point', 1376),
 ('might', 1367),


In [11]:
tf_feature_names = [name for name, count in transformed_dataset]

d_transformed_tokenized = [[word for word in tokenizer.tokenize(doc.lower()) if word in tf_feature_names] for doc in d_prep]

### пункт 2
Получите названия тех топ-1000 слов, которые вы только что закодировали векторайзером

In [12]:
tf_feature_names = [name for name, count in transformed_dataset]
assert len(tf_feature_names) == 1000

Обучите LatentDirichletAllocation из sklearn со следующими параметрами n_topics=20, max_iter=50, learning_method='batch'

In [13]:
n_topics=20

cv = CountVectorizer()

filtered_docs = [' '.join(d) for d in d_transformed_tokenized]
cv.fit(filtered_docs)
bag_of_words = cv.transform(filtered_docs)

In [14]:
lda = LatentDirichletAllocation(n_components=n_topics, learning_method='batch', max_iter=50)

lda.fit(bag_of_words.toarray())

LatentDirichletAllocation(max_iter=50, n_components=20)

Воспользуйтесь следующей функцией, чтобы вывести ключевые слова конкретной темы.

In [15]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [16]:
n_top_words = 10
print_top_words(lda, tf_feature_names, n_top_words)

Topic 0:
49 cut every easy sound news programs 92 1993 talk
Topic 1:
exists sound specific monitor rules certain meaning purpose 55 sl
Topic 2:
anonymous text specific pl trying image mark quite reason meaning
Topic 3:
ax together one might 91 time community current number people
Topic 4:
field knows sell communications technology bad bought claim million hope
Topic 5:
specific sl necessary certainly tom bike sounds keys young 36
Topic 6:
time file good new see first us many right go
Topic 7:
period unless difference million parts single national policy goes everyone
Topic 8:
speed devices required 24 memory perhaps runs three generally entry
Topic 9:
specific rules present rights error chz version therefore limited political
Topic 10:
clear later president inc 1d9 machines court services level generally
Topic 11:
far rate took old 00 two thinking say know really
Topic 12:
plan hell email happen along anybody sources specific 36 graphics
Topic 13:
group gives news provide release sorry

### Метрики: перплексия

In [70]:
lda.perplexity(bag_of_words)

289.2184926372969

### Визуализация

In [18]:
skl_lda = pyLDAvis.lda_model.prepare(lda, bag_of_words, cv)
pyLDAvis.display(skl_lda)

# part 2: Gensim

- Неэффективный.
- Совместим в векторайзерами sklearn.
- Легко использовать.
- Поддерживает несколько входных форматов.

Преобразуйте датасет в корпус с помощью Sparse2Corpus

In [19]:
import gensim

In [46]:
corpus = gensim.matutils.Sparse2Corpus(bag_of_words, documents_columns=False)

### пункт 3
Сформируйте словарь из корпуса и CountVectorizer

In [47]:
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, {i: cv.get_feature_names_out()[i] for i in range(len(cv.get_feature_names_out()))})

common_corpus = [dictionary.doc2bow(text) for text in d_transformed_tokenized]

### пункт 4
Обучите модель LDA из gensim (для этого надо воспользоваться gensim.models.ldamodel.LdaModel)

In [52]:
lda_gensim = gensim.models.ldamodel.LdaModel(corpus, num_topics=n_topics, id2word=dictionary)

Выведите наиболее вероятные слова для каждой темы

In [53]:
lda_gensim.print_topics(lda.n_components)

[(0,
  '0.019*"one" + 0.011*"time" + 0.010*"like" + 0.010*"see" + 0.009*"good" + 0.008*"game" + 0.008*"well" + 0.008*"team" + 0.007*"hockey" + 0.007*"know"'),
 (1,
  '0.016*"sale" + 0.015*"greek" + 0.015*"price" + 0.013*"book" + 0.012*"condition" + 0.012*"new" + 0.011*"never" + 0.011*"offer" + 0.011*"bill" + 0.010*"books"'),
 (2,
  '0.018*"year" + 0.014*"years" + 0.013*"new" + 0.013*"one" + 0.013*"two" + 0.012*"last" + 0.011*"president" + 0.010*"good" + 0.009*"time" + 0.008*"health"'),
 (3,
  '0.026*"drive" + 0.013*"scsi" + 0.013*"card" + 0.013*"system" + 0.013*"disk" + 0.013*"bit" + 0.012*"one" + 0.012*"mac" + 0.010*"hard" + 0.010*"software"'),
 (4,
  '0.013*"image" + 0.012*"window" + 0.012*"windows" + 0.011*"get" + 0.011*"available" + 0.011*"server" + 0.010*"ftp" + 0.010*"files" + 0.010*"program" + 0.009*"system"'),
 (5,
  '0.036*"25" + 0.027*"10" + 0.025*"00" + 0.021*"15" + 0.021*"17" + 0.020*"20" + 0.018*"14" + 0.017*"16" + 0.016*"11" + 0.015*"12"'),
 (6,
  '0.030*"windows" + 0.019

### Метрики: перплексия

In [56]:
lda_gensim.log_perplexity(corpus)

-5.811042783399576

### Визуализация

In [58]:
gsm_lda = pyLDAvis.gensim_models.prepare(lda_gensim, corpus, dictionary)
gsm_lda

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.086057 -0.102577       1        1  12.634289
7     -0.566788 -0.058987       2        1  12.204830
15     0.082304 -0.156379       3        1  11.860681
4      0.067849  0.086913       4        1   7.686261
0      0.088355 -0.089446       5        1   7.133141
19     0.072906  0.042585       6        1   6.361623
3      0.061316  0.060583       7        1   6.307953
2      0.081939 -0.054258       8        1   6.123292
5     -0.058586  0.195680       9        1   4.613637
17     0.074225  0.028202      10        1   4.449084
10     0.077062 -0.123887      11        1   3.809772
18     0.062040 -0.089987      12        1   3.599131
9      0.061667  0.009022      13        1   2.626896
1      0.075278 -0.023929      14        1   2.219082
16     0.022146  0.165683      15        1   2.044502
13     0.064408 -0.042749      16        1   2.013724
6      0.052398  0.051654      17        1   1.913252
12     0.012790  0.145040      18        1   1.122954
11    -0.352176 -0.018648      19        1   0.840534
14    -0.065189 -0.024516      20        1   0.435362, topic_info=       Term          Freq         Total Category  logprob  loglift
142      ax  59485.000000  59485.000000  Default  30.0000  30.0000
5        0d   2004.000000   2004.000000  Default  29.0000  29.0000
574     max   4446.000000   4446.000000  Default  28.0000  28.0000
13      145   1201.000000   1201.000000  Default  27.0000  27.0000
417     god   2569.000000   2569.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
932      us     11.934412   2452.049282  Topic20  -5.4515   0.1115
574     max     12.936705   4446.235306  Topic20  -5.3709  -0.4030
621     new     11.804439   2953.778711  Topic20  -5.4625  -0.0856
536    like     12.237402   4444.779511  Topic20  -5.4265  -0.4582
670  people     11.114363   4290.255225  Topic20  -5.5227  -0.5191

[938 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.010560    00
0         2  0.001056    00
0         3  0.001056    00
0         4  0.013727    00
0         5  0.001056    00
...     ...       ...   ...
998      15  0.005774  york
998      16  0.017323  york
998      17  0.037533  york
998      19  0.002887  york
998      20  0.008661  york

[9885 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 8, 16, 5, 1, 20, 4, 3, 6, 18, 11, 19, 10, 2, 17, 14, 7, 13, 12, 15])

### пункт 5
Обучите модель LSI из gensim (для этого надо воспользоваться gensim.models.lsimodel.LsiModel)

In [60]:
lsi = gensim.models.lsimodel.LsiModel(corpus, num_topics=n_topics, id2word=dictionary)

Выведите наиболее вероятные слова для каждой темы

In [61]:
lsi.print_topics(lsi.num_topics)

[(0,
  '0.997*"ax" + 0.072*"max" + 0.016*"g9v" + 0.012*"b8f" + 0.010*"a86" + 0.009*"pl" + 0.007*"1d9" + 0.006*"1t" + 0.006*"145" + 0.006*"bhj"'),
 (1,
  '0.388*"145" + 0.379*"a86" + 0.375*"b8f" + 0.361*"g9v" + 0.326*"0d" + 0.227*"1d9" + 0.211*"0t" + 0.188*"2di" + 0.173*"_o" + 0.171*"34u"'),
 (2,
  '-0.286*"file" + -0.218*"one" + -0.140*"people" + -0.127*"available" + -0.126*"get" + -0.115*"program" + -0.114*"information" + -0.108*"know" + -0.107*"new" + -0.105*"com"'),
 (3,
  '0.969*"db" + 0.148*"cs" + 0.089*"al" + 0.080*"cx" + 0.063*"bits" + -0.050*"file" + 0.039*"inc" + 0.023*"call" + 0.021*"gas" + 0.020*"higher"'),
 (4,
  '0.675*"g9v" + -0.508*"0d" + -0.332*"_o" + 0.183*"b8f" + -0.128*"145" + -0.128*"6um" + 0.113*"1d9" + -0.110*"6ei" + -0.097*"3t" + -0.088*"75u"'),
 (5,
  '-0.243*"10" + -0.225*"14" + -0.224*"16" + -0.186*"12" + -0.186*"25" + -0.178*"15" + -0.174*"20" + -0.173*"11" + -0.172*"13" + -0.170*"00"'),
 (6,
  '-0.474*"file" + 0.171*"people" + -0.171*"output" + 0.170*"said" 

### Метрики: согласованность

Для измерения согласованности воспользуйтесь gensim.models.CoherenceModel с метрикой coherence='c_v'. В тексты передается токенизированный текст

In [62]:
coherence_model_lsi = gensim.models.CoherenceModel(model=lsi, texts=[d.split(' ') for d in d_prep], coherence='c_v', topn=n_top_words)
coherence_model_lda = gensim.models.CoherenceModel(model=lda_gensim, texts=[d.split(' ') for d in d_prep], coherence='c_v', topn=n_top_words)

In [63]:
coherence_lda = coherence_model_lda.get_coherence()
coherence_lsi = coherence_model_lsi.get_coherence()
print('LDA: ', coherence_lda)
print('LSI: ', coherence_lsi)

LDA:  0.6359269920263455
LSI:  0.6079201112856432


Какая модель по вашему мнению лучше выделила темы?

# part 3: BigARTM

Приведем данные к более удобному для BigARTM формату

In [18]:
with open('fetch20newsgroups.txt', 'w', encoding='utf-8') as file:
    print(*(' |text ' + s for s in d_prep), sep='\n', file=file)

In [ ]:
!cd /content && git clone --branch=stable https://github.com/bigartm/bigartm.git
!cd /content/bigartm && mkdir build
!cd /content/bigartm/build && cmake -DPYTHON=python3 ..
!cd /content/bigartm/build && make
!cd /content/bigartm/build && sudo make install
!export ARTM_SHARED_LIBRARY=/usr/local/lib/libartm.so
!cd /content/bigartm/python && python setup.py install

In [21]:
import artm
import matplotlib.pyplot as plt

ModuleNotFoundError: ignored

Считывание данных из файла происходит через BatchVectorizer. Запись идет в батчи, из них же можно считывать, если у вас есть уже готовые данные в таком формате.
Считайте данные в формате vowpal_wabbit, target_folder="batches", batch_size=100

In [20]:
batch_vectorizer = artm.BatchVectorizer(data_path='',
                                        data_format='vowpal_wabbit',
                                        target_folder='my_collection_batches')

NameError: ignored

Задание словаря - атрибут batch_vectorizer

In [ ]:
dict_artm = batch_vectorizer.dictionary

Для создания модели воспользуйтесь artm.ARTM

In [ ]:
model_artm = 

Задание метрик

In [ ]:
model_artm.scores.add(bigartm.PerplexityScore(name='PerplexityScore', dictionary=dict_artm))
model_artm.scores.add(bigartm.SparsityPhiScore(name='SparsityPhiScore', class_id="text"))
model_artm.scores.add(bigartm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(bigartm.TopTokensScore(name="top_words", num_tokens=n_top_words, class_id="text"))

Обучите модель с помощью метода fit_offline. Проходов по всей коллекции - 50, каждый документ модель видит единожды

In [ ]:
model_artm

Перплексия:

In [ ]:
print(model_artm.score_tracker["PerplexityScore"].last_value)
plt.plot(model_artm.score_tracker["PerplexityScore"].value)

Посмотрим на темы:

In [ ]:
for topic_name, list_tokens in model_artm.score_tracker["top_words"].last_tokens.items():
    print(topic_name+': ', *list_tokens, '\n')

Разреженность матриц:

In [ ]:
print(model_artm.score_tracker["SparsityPhiScore"].last_value)
print(model_artm.score_tracker["SparsityThetaScore"].last_value)

Попробуйте использовать разреживающий регуляризатор для матрицы фи:

In [ ]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-100, dictionary=dict_artm))

In [ ]:
model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

In [ ]:
print(model_artm.score_tracker["SparsityPhiScore"].last_value)
print(model_artm.score_tracker["SparsityThetaScore"].last_value)

In [ ]:
for topic_name, list_tokens in model_artm.score_tracker["top_words"].last_tokens.items():
    print(topic_name+': ', *list_tokens, '\n')

# part 4: BERT

In [64]:
!pip install -U joblib~=1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 KB 16.5 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successfully uninstalled joblib-1.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.0 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.


In [65]:
from bertopic import BERTopic

In [66]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True,)
topics, probs = topic_model.fit_transform(d_prep)

Batches:   0%|          | 0/354 [00:00<?, ?it/s]

2023-03-28 14:31:21,680 - BERTopic - Transformed documents to Embeddings
2023-03-28 14:32:03,102 - BERTopic - Reduced dimensionality
2023-03-28 14:32:10,856 - BERTopic - Clustered reduced embeddings


In [67]:
freq = topic_model.get_topic_info()
freq.head(5)

,Topic,Count,Name
0,-1,3590,-1_com_one_like_get
1,0,1155,0_team_game_hockey_season
2,1,968,1_car_bike_dod_cars
3,2,733,2_god_jesus_bible_christian
4,3,513,3_health_medical_patients_msg


In [68]:
topic_model.get_topic(3)

[('health', 0.01601154117427013),
 ('medical', 0.013477744699221991),
 ('patients', 0.01258645456822608),
 ('msg', 0.011014395626581724),
 ('disease', 0.010339397897003584),
 ('cancer', 0.010268216542836477),
 ('food', 0.010195117794268134),
 ('doctor', 0.010097694546392194),
 ('pain', 0.008956735265707548),
 ('hiv', 0.008205039787902052)]

In [69]:
topic_model.visualize_topics()